# ZVP-based reverse-engineering

In [ ]:
import io
import numpy as np
from sympy import FF, sympify, symbols, Poly, Monomial
from collections import Counter
import tabulate
from functools import partial
from itertools import product
from IPython.display import HTML, display
from tqdm.notebook import tqdm
from anytree import RenderTree, PreOrderIter
from concurrent.futures import ProcessPoolExecutor, as_completed

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters, load_params_ecgen
from pyecsca.ec.formula import FormulaAction, AdditionFormula, DoublingFormula
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod, gcd, SymbolicMod
from pyecsca.sca.re.tree import build_distinguishing_tree, expand_tree
from pyecsca.sca.re.rpa import MultipleContext
from pyecsca.sca.re.zvp import zvp_points, compute_factor_set
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mult import LTRMultiplier, AccumulationOrder
from pyecsca.misc.cfg import getconfig
from pyecsca.ec.error import NonInvertibleError, UnsatisfiedAssumptionError
from pyecsca.sca.re.zvp import unroll_formula, compute_factor_set, zvp_points, addition_chain, precomp_zvp_points

In [ ]:
# TODO: Maybe combine with EPA?

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)

## Exploration
First lets explore the behavior of addition formulas. The following two cells pick a coordinate model along with some formulas and symbolically unroll a scalar multiplication.

In [ ]:
which = "jacobian"
coords = model.coordinates[which]

In [ ]:
getconfig().ec.mod_implementation = "symbolic"
x, y, z = symbols("x y z")

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
field = FF(p)
a = SymbolicMod(Poly(0x4833d7aa73fa6694, x, y, z, domain=field), p)
b = SymbolicMod(Poly(0xa6c44a61c5323f6a, x, y, z, domain=field), p)
gx = SymbolicMod(Poly(0x5fd1f7d38d4f2333, x, y, z, domain=field), p)
gy = SymbolicMod(Poly(0x21f43957d7e20ceb, x, y, z, domain=field), p)
n = 0xc50de885003b80eb
h = 1

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)


add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
mult = LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqRP, True, True)


point = Point(coords,
              X=SymbolicMod(Poly(x, x, y, z, domain=field), params.curve.prime),
              Y=SymbolicMod(Poly(y, x, y, z, domain=field), params.curve.prime),
              Z=SymbolicMod(Poly(z, x, y, z, domain=field), params.curve.prime))
mult.init(params, point)
res = mult.multiply(5)

The result is a Point with coordinates that are polynomials in the input coordinates and curve parameters.

In [ ]:
cfg = getconfig()
cfg.ec.mod_implementation = "gmp"

## Reverse-engineering
Now, lets look at using the ZVP attack for reverse-engineering. First pick 10 random curves, some with $a \in \{0, -1, -3 \}$. The randomcurves are not special in any way and just serve to randomize the process, as the existence of ZVP points for a given intermediate value polynomial depends on the curve.

In [ ]:
curves = list(map(lambda spec: load_params_ecgen(io.BytesIO(spec.encode()), "affine"), [
    # Random
    """[{"field":{"p":"0xa7ec3617d4166b2d"},"a":"0x372994d9d680a83b","b":"0xa0a2bf719d8e68c5","order":"0xa7ec3618be1dab55","subgroups":[{"x":"0x1ef15756946a5b6d","y":"0x2ca9658f7ab9a558","order":"0xa7ec3618be1dab55","cofactor":"0x1","points":[{"x":"0x1ef15756946a5b6d","y":"0x2ca9658f7ab9a558","order":"0xa7ec3618be1dab55"}]}]}]""",
    """[{"field":{"p":"0xa42c1467a1ed04f3"},"a":"0x55d07340a4572f2d","b":"0x0a938c37dfb0b6d5","order":"0xa42c14689284d3a7","subgroups":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7","cofactor":"0x1","points":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7"}]}]}]""",
    """[{"field":{"p":"0xea0d9cead19016ab"},"a":"0xcbbfe501c4ef6d92","b":"0x5762de777a6d9178","order":"0xea0d9cea8cd2c857","subgroups":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857","cofactor":"0x1","points":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857"}]}]}]""",
    #"""[{"field":{"p":"0x9c7e7216decb71a7"},"a":"0x324ef48887401a87","b":"0x3ce6f35a00280102","order":"0x9c7e72175ebfe709","subgroups":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709","cofactor":"0x1","points":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709"}]}]}]""",
    #"""[{"field":{"p":"0xeb5779f0bbf1ef5b"},"a":"0x2419e8bbc7b5f8f2","b":"0xe74e5d3064a4f2e3","order":"0xeb5779f21320c2e9","subgroups":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9","cofactor":"0x1","points":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9"}]}]}]""",
    #"""[{"field":{"p":"0x97b6ea097868b95d"},"a":"0x550a41d65e4bcd13","b":"0x47c5e527113b261c","order":"0x97b6ea094947a76b","subgroups":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b","cofactor":"0x1","points":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b"}]}]}]""",
    #"""[{"field":{"p":"0xa00629e6522032f7"},"a":"0x896f04a7ae302922","b":"0x6bc03365b1f1cb50","order":"0xa00629e5c03cf913","subgroups":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913","cofactor":"0x1","points":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913"}]}]}]""",
    #"""[{"field":{"p":"0xd47ec1d03a62686d"},"a":"0xd00a3ee0f5c86b02","b":"0x457a5b6c47db38d8","order":"0xd47ec1d107db7d6f","subgroups":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f","cofactor":"0x1","points":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f"}]}]}]""",
    #"""[{"field":{"p":"0xb1c9115c6f40d755"},"a":"0x79d3ceefafc44ce9","b":"0x8316af84264df42b","order":"0xb1c9115d17f84a45","subgroups":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45","cofactor":"0x1","points":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45"}]}]}]""",
    #"""[{"field":{"p":"0x8f738fda18cd5dff"},"a":"0x4747f2f9b8628cbf","b":"0x586cdb9378a1389f","order":"0x8f738fd8fc7ebed3","subgroups":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3","cofactor":"0x1","points":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3"}]}]}]""",
    # a = -1
    """[{"field":{"p":"0xcfef393139c3007f"},"a":"0xcfef393139c3007e","b":"0x950312812acb155f","order":"0xcfef39320179387b","subgroups":[{"x":"0xae2d2f58ca5b5cf7","y":"0xc3a4bf3a1dc10005","order":"0xcfef39320179387b","cofactor":"0x1","points":[{"x":"0xae2d2f58ca5b5cf7","y":"0xc3a4bf3a1dc10005","order":"0xcfef39320179387b"}]}]}]""",
    """[{"field":{"p":"0xb0461c0e4946cbd5"},"a":"0xb0461c0e4946cbd4","b":"0x082c3722016def51","order":"0xb0461c0e07e3e1bf","subgroups":[{"x":"0x5142200263be1fe3","y":"0x14984b7551ed21a9","order":"0xb0461c0e07e3e1bf","cofactor":"0x1","points":[{"x":"0x5142200263be1fe3","y":"0x14984b7551ed21a9","order":"0xb0461c0e07e3e1bf"}]}]}]""",
    """[{"field":{"p":"0xeff607c2dc4f278b"},"a":"0xeff607c2dc4f278a","b":"0x26fd03674f5092d2","order":"0xeff607c30ab8c50d","subgroups":[{"x":"0x004d4a5a9bb849fe","y":"0x80eb7ef89110c149","order":"0xeff607c30ab8c50d","cofactor":"0x1","points":[{"x":"0x004d4a5a9bb849fe","y":"0x80eb7ef89110c149","order":"0xeff607c30ab8c50d"}]}]}]""",
    # a = -3
    """[{"field":{"p":"0x8d79ca36cee026a7"},"a":"0x8d79ca36cee026a4","b":"0x0478c1f80ce2c9c6","order":"0x8d79ca35a428c76f","subgroups":[{"x":"0x2e94a3e38f8b345e","y":"0x83e6c6f0cb8f69c4","order":"0x8d79ca35a428c76f","cofactor":"0x1","points":[{"x":"0x2e94a3e38f8b345e","y":"0x83e6c6f0cb8f69c4","order":"0x8d79ca35a428c76f"}]}]}]""",
    """[{"field":{"p":"0xfcada438caa11847"},"a":"0xfcada438caa11844","b":"0x08dfa4498ff16fda","order":"0xfcada437b9db629d","subgroups":[{"x":"0xf04a2a334193d501","y":"0x364b3b33e3e200e9","order":"0xfcada437b9db629d","cofactor":"0x1","points":[{"x":"0xf04a2a334193d501","y":"0x364b3b33e3e200e9","order":"0xfcada437b9db629d"}]}]}]""",
    """[{"field":{"p":"0xcb5aa8a7a10aa06b"},"a":"0xcb5aa8a7a10aa068","b":"0x31fe9c57c570174f","order":"0xcb5aa8a6cf812191","subgroups":[{"x":"0x84c75d46fc687ff1","y":"0x7424362ac73df187","order":"0xcb5aa8a6cf812191","cofactor":"0x1","points":[{"x":"0x84c75d46fc687ff1","y":"0x7424362ac73df187","order":"0xcb5aa8a6cf812191"}]}]}]""",
    # a = 0
    """[{"field":{"p":"0xceaf446a53f14bc1"},"a":"0x0000000000000000","b":"0x326539376260f173","order":"0xceaf446aae275419","subgroups":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419","cofactor":"0x1","points":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419"}]}]}]""",
    """[{"field":{"p":"0xb3c2beca75d66de3"},"a":"0x0000000000000000","b":"0x46069225826b51aa","order":"0xb3c2bec95881b695","subgroups":[{"x":"0x81500c226efa0d5a","y":"0x674e09d296452eee","order":"0xb3c2bec95881b695","cofactor":"0x1","points":[{"x":"0x81500c226efa0d5a","y":"0x674e09d296452eee","order":"0xb3c2bec95881b695"}]}]}]""",
    """[{"field":{"p":"0xd6097c1ce207aae7"},"a":"0x0000000000000000","b":"0x7adaab54e7dfd564","order":"0xd6097c1b407eb413","subgroups":[{"x":"0x151da8fb1f83201e","y":"0x8bfeb90ec1177a91","order":"0xd6097c1b407eb413","cofactor":"0x1","points":[{"x":"0x151da8fb1f83201e","y":"0x8bfeb90ec1177a91","order":"0xd6097c1b407eb413"}]}]}]""",
    # b = 0 (causes more issues than gain)
    #"""[{"field":{"p":"0x9d9119957f02fe3f"},"a":"0x0106903196d88df9","b":"0x0000000000000000","order":"0x9d9119957f02fe40","subgroups":[{"x":"0x191a36b9cd81de96","y":"0x10f2c6bded391aa9","order":"0x9d9119957f02fe40","cofactor":"0x1","points":[{"x":"0x0000000000000000","y":"0x0000000000000000","order":"0x2"},{"x":"0x95913fae9065da0f","y":"0x5eeddeee7152d6fb","order":"0x276446655fc0bf9"}]}]}]"""
]))

First lets fix some scalars, go over the curves and compute the addition chain to obtain information about which multiples of the input point will go into the formulas.

In [ ]:
scalars = [123456789, 98765432, 0b10000000, 0b1010101010, 0b1010101, 77777, 66666, 55555, 44444, 33333, 22222, 11111, 0b1111111111]

chains = []
scalar_map = {}
for i, (scalar, params) in enumerate(zip(scalars, curves)):
    chain = addition_chain(scalar, params, LTRMultiplier, lambda add,dbl: LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqRP, True, True))
    chains.append(chain)
    scalar_map[params] = scalar

Now, lets compute the sets of ZVP points, going over all coordinate systems and all of their formulas (that fit the scalar multiplier) and store them into the `point_chains`. These items form individual distinct entries of the distinguishing table.

In [ ]:
# bound is the maximal dlog in the hard case of the DCP to be solved
bound = 15
# chain_bound is the number of formula applications at the start of the addition chain that is processed
chain_bound = 30

formula_classes = [AdditionFormula, DoublingFormula]
point_chains = {}
with ProcessPoolExecutor(max_workers=30) as pool:
    futures = []
    args = []
    for coord_name, coords in model.coordinates.items():
        for chain, affine_params in zip(chains, curves):
            try:
                params = affine_params.to_coords(coords)
            except UnsatisfiedAssumptionError:
                continue
            formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]

            for formula_group, formula_class in zip(formula_groups, formula_classes):
                for formula in formula_group:
                    futures.append(pool.submit(precomp_zvp_points, chain[:chain_bound], {formula_class.shortname: formula}, params, bound, filter_nonhomo=False))
                    args.append((coords, formula, affine_params))
    for future in tqdm(as_completed(futures), desc="Compute", total=len(futures)):
        j = futures.index(future)
        point_chains[args[j]] = future.result()

Now, accumulate the rows of the distinguishing table to create the distinguishing map `point_map`.

In [ ]:
point_map = {}
for coord_name, coords in tqdm(model.coordinates.items(), desc="Accumulate for coord systems"):
    formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
    formula_combinations = list(product(*formula_groups))
    for formulas in formula_combinations:
        points = set()
        for formula in formulas:
            for chain, affine_params in zip(chains, curves):
                try:
                    params = affine_params.to_coords(coords)
                except UnsatisfiedAssumptionError:
                    continue
                if (coords, formula, affine_params) not in point_chains:
                    print(f"Missing {formula}")
                    continue
                point_chain = point_chains[(coords, formula, affine_params)]
                for step in point_chain:
                    for poly, poly_points in step.items():
                        for point in poly_points:
                            points.add((point, affine_params))
        point_map[formulas] = points
all_points = set().union(*point_map.values())
print(len(all_points))

We now have a distinguishing map so we can build the tree and visualize it.

In [ ]:
tree = build_distinguishing_tree(set(point_map.keys()), point_map)

In [ ]:
def render_tree(tree):
    print(RenderTree(tree).by_attr(lambda n: n.name if n.name is not None else "\n".join((", ".join(str(formula) for formula in cfg) for cfg in n.cfgs))))

def describe_tree(tree):
    leaf_sizes = [len(leaf.cfgs) for leaf in tree.leaves]
    print(f"Total cfgs: {len(tree.cfgs)}")
    print(f"Depth: {tree.height}")
    print(f"Leaf sizes: {sorted(leaf_sizes)}")
    print(f"Average leaf size: {np.mean(leaf_sizes):.3}")
    leafs = sum(([size] * size for size in leaf_sizes), [])
    print(f"Mean result size: {np.mean(leafs):.3}")

In [ ]:
describe_tree(tree)

In [ ]:
render_tree(tree)

Our ZVP points might (due to the bounds above thus the incompleteness of our analysis) lead to more zeros than we attribute to them (in more configurations), i.e. "false negatives". They might also be erroneous and not lead to zeros if the argument `filter_nonhomo` is False, as non-homogenous intermediate polynomials are not filtered out of the analysis. They introduce "false positives" but also some true positives.

Thus we perform a remapping step where we execute the scalar multiplication with given points and trace whether they introduce the zeros. This gives us a new distinguishing map `remapped_hit_point_map`, now without "false negatives" or "false positives".

Note that we also construct two additional maps `remapped_count_point_map` and `remapped_position_point_map` which represent the results of remapping using an oracle counting the zeros and an oracle giving the positions of the zeros in the computation, respectively.

In [ ]:
def remap(coords, formulas, points, scalar_map):
    mult = LTRMultiplier(*formulas, None, False, AccumulationOrder.PeqRP, True, True)
    hit_points = set()
    count_points = set()
    position_points = set()
    
    param_map = {}
    for point, params in tqdm(points):
        if params not in param_map:
            try:
                param_map[params] = params.to_coords(coords)
            except UnsatisfiedAssumptionError:
                param_map[params] = None
                continue
        elif param_map[params] is None:
            continue
        mult.init(param_map[params], point.to_model(param_map[params].curve.coordinate_model, param_map[params].curve))
        scalar = scalar_map[params]
        with local(DefaultContext()) as ctx:
            try:
                mult.multiply(scalar)
            except UnsatisfiedAssumptionError:
                continue
        trace = []
        def callback(action):
            if isinstance(action, FormulaAction):
                for intermediate in action.op_results:
                    trace.append(intermediate.value)
        ctx.actions.walk(callback)
        zeros = tuple(map(lambda x: int(x) == 0, trace))
        if any(zeros):
            hit_points.add((point, params))
            count_points.add((point, sum(zeros), params))
            position_points.add((point, zeros, params))
    return hit_points, count_points, position_points

remapped_hit_point_map = {}
remapped_count_point_map = {}
remapped_position_point_map = {}
with ProcessPoolExecutor(max_workers=30) as pool:
    futures = []
    pairs = []
    for coord_name, coords in model.coordinates.items():
        formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
        formula_combinations = list(product(*formula_groups))
        for formulas in formula_combinations:
            futures.append(pool.submit(remap, coords, formulas, all_points, scalar_map))
            pairs.append(formulas)
    results = [None for _ in futures]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Remapping"):
        j = futures.index(future)
        cfg = pairs[j]
        h, c, p = future.result()
        remapped_hit_point_map[cfg] = h 
        remapped_count_point_map[cfg] = c
        remapped_position_point_map[cfg] = p

We can compare the remapped distinguishing map to the original one and see the changes.

In [ ]:
table = [["Add", "Dbl", "raw", "remapped", "removed", "new"]]
for pair in point_map.keys():
    table.append((pair[0], pair[1],
          len(point_map[pair]),
          len(remapped_hit_point_map[pair]),
          -len(point_map[pair].difference(remapped_hit_point_map[pair])),
          len(remapped_hit_point_map[pair].difference(point_map[pair]))))

display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

Finally, we can build a tree using the remapped distinguishing map.

In [ ]:
remapped_tree = build_distinguishing_tree(set(remapped_hit_point_map.keys()), remapped_hit_point_map)

In [ ]:
describe_tree(remapped_tree)

We can also investigate the other oracles and the distinguishing trees they can build:

In [ ]:
remapped_count_tree = build_distinguishing_tree(set(remapped_count_point_map.keys()), remapped_count_point_map)
remapped_position_tree = build_distinguishing_tree(set(remapped_position_point_map.keys()), remapped_position_point_map)

In [ ]:
print("Zero hit")
describe_tree(remapped_tree)
print("\nZero count")
describe_tree(remapped_count_tree)
print("\nZero position")
describe_tree(remapped_position_tree)

Now, lets examine the representation of factor sets and the distinguishing trees they can build.

In [ ]:
fset_map = {}
fset_nonhomo_map = {}
for coord_name, coords in model.coordinates.items():
    formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
    factor_sets = {}
    factor_sets_nonhomo = {}
    for formula_group in formula_groups:
        for formula in formula_group:
            factor_sets[formula] = compute_factor_set(formula)
            factor_sets_nonhomo[formula] = compute_factor_set(formula, filter_nonhomo=False)
    formula_combinations = list(product(*formula_groups))
    for formulas in formula_combinations:
        fset = set()
        fset_nonhomo = set()
        for formula in formulas:
            fset.update(factor_sets[formula])
            fset_nonhomo.update(factor_sets_nonhomo[formula])
        fset_map[formulas] = fset
        fset_nonhomo_map[formulas] = fset_nonhomo

In [ ]:
fset_tree = build_distinguishing_tree(set(fset_map.keys()), fset_map)
fset_nonhomo_tree = build_distinguishing_tree(set(fset_nonhomo_map.keys()), fset_nonhomo_map)

In [ ]:
print("Factor sets")
describe_tree(fset_tree)
print("\nFactor sets (nonhomogenous)")
describe_tree(fset_nonhomo_tree)